In [32]:
import pandas as pd 
import chromadb
import os

chroma_path ="../chroma_db"

df = pd.read_pickle("../data/embedded_data.pkl")



In [33]:
df["id"] = "tweet" + df.index.astype(str)
df.head()

,airline_sentiment,text,text_normalized,embedding,id
0,neutral,@VirginAmerica What @dhepburn said.,said,"[0.0030890966299921274, 0.08647377043962479, -...",tweet0
1,positive,@VirginAmerica plus you've added commercials t...,plus youve added commercials experience tacky,"[-0.017043905332684517, 0.04139120131731033, 0...",tweet1
2,neutral,@VirginAmerica I didn't today... Must mean I n...,didnt today must mean need take another trip,"[0.017651421949267387, -0.0018262050580233335,...",tweet2
3,negative,@VirginAmerica it's really aggressive to blast...,really aggressive blast obnoxious entertainmen...,"[0.05205097422003746, 0.008738746866583824, 0....",tweet3
4,negative,@VirginAmerica and it's a really big bad thing...,really big bad thing,"[-0.027435963973402977, 0.06341183185577393, -...",tweet4


<h1 style="color:orange;"> Creating collection ! ! </h1>



In [34]:
client = chromadb.PersistentClient(path=chroma_path)

try:
    client.delete_collection(name="tweets")
    print("the collectios was deleted")
except:
    print("pass")
    pass

# metadata={"hnsw:space": "cosine"} tells Chroma to use Cosine Similarity for search

collection = client.create_collection(name="tweets", metadata={"hnsw:space": "cosine"})


the collectios was deleted


In [35]:
# We will use a loop (batching) to avoid crashing the computer. We map the DataFrame columns to the 3 ChromaDB parts (ids, embeddings, metadatas)

from tqdm import tqdm

def add_to_collection(collection, df, batch_size=2000):
    total_rows = len(df)
    # Loop from 0 to total_rows in steps of 2000
    
    for i in tqdm(range(0, total_rows, batch_size),desc=f'add to {collection.name}'):
        
        batch = df.iloc[i : i+ batch_size]
        
        ids = batch['id'].tolist()
        
        embeddings = batch["embedding"].tolist()
        
        metadatas = batch[["airline_sentiment", "text_normalized"]].to_dict("records")
        
        collection.add(
            ids=ids,
            embeddings=embeddings,
            metadatas=metadatas
        )

In [36]:
print("Starting ingestion for Data...")

add_to_collection(collection, df)

Starting ingestion for Data...


add to tweets: 100%|██████████| 14/14 [00:11<00:00,  1.18it/s]
